In [141]:
!pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [142]:
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
  Preparing metadata (setup.py) ... done


In [143]:
!pip install pyjanitor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [144]:
!pip install mlxtend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [145]:
!pip install six

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Sobre o Trabalho

**Problema 1**

A empresa gostaria de saber se é possível traçar alguma estratégia comum para cada grupo de cliente, no entento essa análise nunca foi feita e ninguém, até o momento, sabe ao certo qual seria a melhor forma de conseguir realizar esse tipo de análise. Hoje, no conjunto de dados, não existem informações claras relativas ao segmento do cliente ou tamanho, apenas dados relativos ao faturamento, ano de fundação, edividamento e etc.

Isso ajudario muito os times de marketing e análise de crédito.

**Problema 2**

Um problema já mapeado é o que fzer após a geração desses grupos quando chegar um novo cliente solicitando um novo crédito?

COmo seria possível indicar o grupo desse liente sem ter o que refazer todos os grupos?

**Problema 3**

A empresa deseja, entre outras coisas, criar um "robô automatizado" (na linguagem da própria empresa), que realize uma recomendação de crédito para os analistas se baseiem nessa recomendação para conceder o limite máximo de crédito.

Vale salientar qeue por se tratar de valores monetários é importante que este modeo tenha um bom desempenho;

### Bibliotecas utilizadas

*   Pandas - Manipulação dos dados
*   Sickit-learn - modelagem preditiva
*   Yellowbrick - visuliazação e avaliação de modelos



In [146]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import (
    ensemble,
    preprocessing,
    tree
)
from sklearn.metrics import (
    auc,
    confusion_matrix,
    roc_auc_score,
    roc_curve
)
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold
)
from yellowbrick.classifier import (
    ConfusionMatrix,
    ROCAUC
)
from yellowbrick.model_selection import (
    LearningCurve
)

### Importação da base

Importamos a nossa base **titanic.csv** que contém as seguintes colunas:

*   numero_solicitacao: Número da solicitação; 
*   razaoSocial: Razão Social Anonimizado;
* cnpjSemTraco: CNPJ Anonimizado;
* maiorAtraso: Maior atraso de pagamento em dias;
* margemBrutaAcumulada: Margem bruta acumulada;
* percentualProtestos: Percentual protestos;
* primeiraCompra: Data da primeira compra na Wtec;
* prazoMedioRecebimentoVendas: prazo médio do recebimento de vendas do cliente;
* titulosEmAberto: valor total de títulos em aberto;
* valorSolicitado; Valor de crédito solicitado;


In [147]:
clientes_df = pd.read_csv("solicitacoescredito.csv", sep=",")
original_df = clientes_df
clientes_df.head()

,numero_solicitacao,razaoSocial,nomeFantasia,cnpjSemTraco,maiorAtraso,margemBrutaAcumulada,percentualProtestos,primeiraCompra,prazoMedioRecebimentoVendas,titulosEmAberto,...,periodoDemonstrativoEmMeses,custos,anoFundacao,intervaloFundacao,capitalSocial,restricoes,empresa_MeEppMei,scorePontualidade,limiteEmpresaAnaliseCredito,dataAprovadoNivelAnalista
0,1,James Richardson-Patel,Alexandra Williams,KEBE17609492220843,0,0.252448,0.0,2015-12-10T00:00:00,0,0.00,...,12.0,0.0,2003.0,Acima de 17 anos,90000.0,False,True,1.0,43200.0,2020-02-03T20:57:33
1,2,Dr. Geoffrey Walsh,Mr. Darren Arnold,JRBK88908250677300,0,0.000000,NaN,2019-06-12T17:28:31,0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,3,Joanna Hudson,Dr. David Rees,GCVQ28531614261293,4,0.624777,0.0,2019-11-27T00:00:00,0,0.00,...,7.0,0.0,2014.0,De 6 a 10 anos,20000.0,False,True,1.0,4320.0,2020-02-04T16:40:49
3,4,Gordon Jones-Hopkins,Sara Reid-Robson,KJND32266018316396,20,0.000000,NaN,2017-02-13T17:20:27,0,0.00,...,12.0,0.0,2013.0,De 6 a 10 anos,30000.0,False,True,0.0,5920.0,2020-02-04T16:37:52
4,5,Nigel Lee,Dr. Stanley Duncan,CGQN15826802440348,20,0.454088,0.0,2010-07-13T00:00:00,20,1486.95,...,9.0,40680051.0,2002.0,Acima de 17 anos,75000.0,False,False,1.0,89000.0,2020-02-04T15:06:28


### Limpeza dos dados

Antes da limpeza verificamos os dados do nosso modelo:

In [148]:
clientes_df.dtypes

numero_solicitacao               int64
razaoSocial                     object
nomeFantasia                    object
cnpjSemTraco                    object
maiorAtraso                      int64
margemBrutaAcumulada           float64
percentualProtestos            float64
primeiraCompra                  object
prazoMedioRecebimentoVendas      int64
titulosEmAberto                float64
valorSolicitado                float64
status                          object
definicaoRisco                  object
diferencaPercentualRisco       float64
percentualRisco                float64
dashboardCorrelacao            float64
valorAprovado                  float64
dataAprovadoEmComite            object
periodoBalanco                  object
ativoCirculante                float64
passivoCirculante              float64
totalAtivo                     float64
totalPatrimonioLiquido         float64
endividamento                  float64
duplicatasAReceber             float64
estoque                  

A seguir utilizamos o pandas_profiling que irá sintetizar os tipos das colunas e permite que sejamos capazes se visualizar os detalhes estatísticas dos quantis, as estatísticas descritivas, um histograma os o valoes comuns.

In [149]:
from pandas_profiling import ProfileReport

In [150]:
# Número de linhas e colunas
clientes_df.shape

(8973, 38)

In [151]:
profile = ProfileReport(clientes_df.head(100), title='SolicitacaoCredito', html={'style':{'full_width':True}})

In [152]:
profile.to_notebook_iframe()

Output hidden; open in https://colab.research.google.com to view.

### Verificação de dados nulos

Aqui nós vamos verificar os dados nulos do nosso modelo, além disso colocamos o resultado em um DataFrame para exibir o total encontrado X a proporção de valores nulos do nosso modelo.

In [153]:
clientes_df.isnull().mean(axis=1).loc[:10]

0     0.236842
1     0.578947
2     0.236842
3     0.263158
4     0.026316
5     0.263158
6     0.263158
7     0.026316
8     0.026316
9     0.552632
10    0.052632
dtype: float64

In [154]:
clientes_df.isnull() # Aqui verificamos o total de null na base se existir
clientes_df.isnull().sum()/len(clientes_df) # Aqui obtemos a proporção de null se existir

# Criamos um novo DataFrame para verificar o total e a proporção de null na base
pd.DataFrame(zip(clientes_df.isnull().sum(),clientes_df.isnull().sum()/len(clientes_df)), columns=['Count', 'Proportion'], index=clientes_df.columns)

,Count,Proportion
numero_solicitacao,0,0.000000
razaoSocial,0,0.000000
nomeFantasia,0,0.000000
cnpjSemTraco,0,0.000000
maiorAtraso,0,0.000000
margemBrutaAcumulada,0,0.000000
percentualProtestos,1498,0.166945
primeiraCompra,106,0.011813
prazoMedioRecebimentoVendas,0,0.000000
titulosEmAberto,0,0.000000


### Problema 1

A empresa gostaria de saber se é possível traçar alguma estratégia comum para cada grupo de cliente, no entento essa análise nunca foi feita e ninguém, até o momento, sabe ao certo qual seria a melhor forma de conseguir realizar esse tipo de análise. Hoje, no conjunto de dados, não existem informações claras relativas ao segmento do cliente ou tamanho, apenas dados relativos ao faturamento, ano de fundação, edividamento e etc.

Isso ajudario muito os times de marketing e análise de crédito.

In [155]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [156]:
dados = clientes_df

In [157]:
dados.fillna(0, inplace = True)

In [158]:
dados.isnull().sum()

numero_solicitacao             0
razaoSocial                    0
nomeFantasia                   0
cnpjSemTraco                   0
maiorAtraso                    0
margemBrutaAcumulada           0
percentualProtestos            0
primeiraCompra                 0
prazoMedioRecebimentoVendas    0
titulosEmAberto                0
valorSolicitado                0
status                         0
definicaoRisco                 0
diferencaPercentualRisco       0
percentualRisco                0
dashboardCorrelacao            0
valorAprovado                  0
dataAprovadoEmComite           0
periodoBalanco                 0
ativoCirculante                0
passivoCirculante              0
totalAtivo                     0
totalPatrimonioLiquido         0
endividamento                  0
duplicatasAReceber             0
estoque                        0
faturamentoBruto               0
margemBruta                    0
periodoDemonstrativoEmMeses    0
custos                         0
anoFundaca

In [159]:
# Selecionar as colunas relevantes
cols = ['maiorAtraso', 'margemBrutaAcumulada', 'percentualProtestos', 'prazoMedioRecebimentoVendas', 'titulosEmAberto', 'valorSolicitado']

# Normalizar os dados
scaler = StandardScaler()
dados_norm = scaler.fit_transform(dados[cols])

In [160]:
# Executar o algoritmo de K-Means com 4 clusters
kmeans = KMeans(n_clusters=4, random_state=0).fit(dados_norm)

# Adicionar os rótulos dos clusters aos dados
clientes_df['cluster'] = kmeans.labels_

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [161]:
# Visualizar os clusters e suas características médias
clientes_df.groupby('cluster').mean()

,numero_solicitacao,maiorAtraso,margemBrutaAcumulada,percentualProtestos,prazoMedioRecebimentoVendas,titulosEmAberto,valorSolicitado,diferencaPercentualRisco,percentualRisco,dashboardCorrelacao,...,duplicatasAReceber,estoque,faturamentoBruto,margemBruta,periodoDemonstrativoEmMeses,custos,anoFundacao,capitalSocial,scorePontualidade,limiteEmpresaAnaliseCredito
cluster,,,,,,,,,,,,,,,,,,,,,
0,4679.072405,21.645171,0.455057,0.003902,28.918526,83111.014054,4.663325e+05,0.751019,0.248981,0.045911,...,4.444804e+06,4.565699e+06,3.332229e+07,1.012627e+07,9.554196,1.539263e+07,1849.701742,7.612855e+06,0.965043,2.078707e+06
1,2912.166667,28.000000,0.273985,18.912379,89.166667,0.000000,2.416667e+04,0.761624,0.238376,0.433013,...,0.000000e+00,0.000000e+00,2.319856e+08,0.000000e+00,5.666667,0.000000e+00,1335.166667,8.333333e+04,1.000000,4.926000e+04
2,4111.510902,34.937066,0.042629,0.001679,2.757185,1778.734286,3.881098e+05,0.747866,0.252134,0.050188,...,2.364021e+07,1.995792e+07,1.119704e+08,3.100067e+07,9.368186,6.217683e+07,1805.589197,1.951175e+07,0.223698,4.462852e+06
3,3754.500000,39.000000,0.427106,0.000000,64.000000,562919.690000,1.350000e+09,0.767857,0.232143,0.514470,...,7.228409e+07,6.791021e+07,7.189003e+08,1.936425e+08,10.500000,5.252578e+08,2013.000000,5.700000e+06,0.988907,5.527710e+06


### Problema 2

Com o problema já mapeado é o que fazer após a geração desses grupos
quando chegar um novo cliente solicitando um novo crédito? 
como seria possível indicar o grupo desse cliente sem ter que 
refazer todos os grupos?

In [162]:
import numpy as np

In [163]:
novo_cliente_norm = scaler.transform(dados[cols])

novo_cliente_norm

array([[-0.37237341, -0.5447063 , -0.02961522, -0.33859063, -0.26129078,
        -0.03091605],
       [-0.37237341, -1.79789862, -0.02961522, -0.33859063, -0.26129078,
        -0.0326846 ],
       [-0.31192955,  1.30359208, -0.02961522, -0.33859063, -0.26129078,
        -0.03224246],
       ...,
       [-0.32704052,  0.10793546, -0.02961522, -0.27991711, -0.24947556,
        -0.0264947 ],
       [-0.20615278,  0.30739488, -0.02961522,  0.11612918,  1.99301707,
         0.02390877],
       [-0.31192955,  0.01422208, -0.02961522,  0.4828387 , -0.20635085,
        -0.03091605]])

In [164]:
# Calcular a distância euclidiana do novo cliente para cada centroide
distancias = [np.linalg.norm(novo_cliente_norm - centroide) for centroide in dados_norm]

# Atribuir o novo cliente ao grupo com centroide mais próximo
novo_cliente_grupo = np.argmin(distancias)

# Imprimir o grupo atribuído ao novo cliente
print("O novo cliente pertence ao grupo:", novo_cliente_grupo)

O novo cliente pertence ao grupo: 341


### Problema 3

A empresa deseja, entre outras coisas, criar um robô automatizado
na linguagem da própria empresa, que realiza um recomendação
de crédito para que os analistas se baseiem nessa recomendação
para conceder o limite máximo de crédito.

In [183]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

# Carrega os dados em um DataFrame
data = original_df

# Seleciona as variáveis relevantes para o modelo
features = ['maiorAtraso', 'margemBrutaAcumulada', 'percentualProtestos', 'prazoMedioRecebimentoVendas',
            'titulosEmAberto', 'valorSolicitado', 'diferencaPercentualRisco', 'percentualRisco',
            'dashboardCorrelacao', 'ativoCirculante', 'passivoCirculante', 'totalAtivo',
            'totalPatrimonioLiquido', 'endividamento', 'duplicatasAReceber', 'estoque', 'faturamentoBruto',
            'margemBruta', 'periodoDemonstrativoEmMeses', 'custos', 'anoFundacao', 'scorePontualidade',
            'limiteEmpresaAnaliseCredito', 'cluster', 'valorAprovado']

# Filtra as variáveis selecionadas
data = data[features + ['status']]

# Codifica as variáveis categóricas
encoder = LabelEncoder()
data['status'] = encoder.fit_transform(data['status'])

# Separa os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(data.drop('status', axis=1), data['status'], test_size=0.2, random_state=42)

# Cria o modelo de classificação
model = RandomForestClassifier()

# Treina o modelo com os dados de treino
model.fit(X_train, y_train)

# Realiza a previsão com os dados de teste
y_pred = model.predict(X_test)

# Avalia o desempenho do modelo com a métrica de acurácia
accuracy = (y_pred == y_test).sum() / len(y_test)
print('Acurácia do modelo: {:.2f}%'.format(accuracy * 100))

<ipython-input-183-4403b10e3b18>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['status'] = encoder.fit_transform(data['status'])


Acurácia do modelo: 91.53%
